
---


Integrantes:
*   Oziel Banda Hernández
*   Joshua Santiago Cruz Pérez
*   Naomi Daniela Jiménez Borzani
*   Ximena Paredes Hernández


---








---


25. Para mostrar la diferencia numérica entre el método de las Ecuaciones Normales y la factorización $Q R$ se requiere un problema de mínimos cuadrados que sea mal condicionado. Para ello consideremos el ajuste del siguiente polinomio de grado $n-1$,

$$
p_{n-1}(t)=x_1+x_2 t+x_3 t^2+\cdots+x_n t^{n-1}
$$

para $m$ datos $\left(t_i, y_i\right), m>n$. Elegimos $t_i=(i-1) /(m-1), i=1, \cdots, m$. Los valores para $y_i$ serán los dados al evaluar el polinomio con las $t_i$ dadas previamente y tomando $x_j=1$ para $j=1, \cdots, n$, se quiere ver si se pueden recuperar los valores de las $x_j$ con los métodos estudiados.


Primero se genera un perturbación para los valores $y_i$ dado por
$$
y_i=y_i+\left(2 u_i-1\right) * \epsilon, \quad i=1, \cdots, m
$$
con $u_i \in[0,1]$ números aleatorios. Usar $m=21, n=12$ y $\epsilon=10^{-10}$.

Después de generar la lista de datos $\left(t_i, y_i\right)$ se comparan los dos métodos para ajustar el polinomio. Primero, formar el Sistema de Ecuaciones Normales para el problema y resolverlo por la Factorización de Cholesky. ${ }^1$ Segundo utilizar algún método para factorizar la matriz en una $Q R$ y resolver.

a) ¿Para cuál de los métodos la solución es más sensible a la perturbación generada?.

b) ¿Cuál de los métodos está más proximo a tener la solución exacta dada por $x_i=1$ ?.

c) ¿La diferencia en las soluciones afecta en el ajuste de puntos $\left(t_i, y_i\right)$ por el polinomio, por qué?.

Argumentar todas las respuestas con los resultados de ambos métodos.



---



In [ ]:
import numpy as np
from numpy import linalg as LA
# =============================================
# Funciones previas para Cholesky
# =============================================
def SustDelante(L,b):
  x=np.zeros_like(b)
  n=L.shape[0]# cantidad de renglones de L
  for i in range(n):
    sum=0.0
    for j in range(i):
      sum+=L[i,j]*x[j]
    x[i]=(b[i]-sum)/L[i,i]
  return x

def SustAtras(U,y):
    x=np.zeros_like(y)
    n=U.shape[0]# cantidad de renglones de U
    x[n-1] = y[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        sum=0.0
        for j in range(i+1,n):
            sum+=U[i,j]*x[j]
        x[i]=(y[i]-sum)/U[i,i]
    return x

def Cholesky(A):
    n=A.shape[0]
    L=np.zeros_like(A)

    for i in range(n):
      for j in range(i+1):
        if i==j:
          sum=0.0
          for k in range(i):
            sum+= L[i][k]*L[i][k]
          L[i][i]=np.sqrt(A[i][i]-sum)

        else:
          sum=0.0
          for k in range(j):
            sum+= L[i][k]*L[j][k]
          L[i][j]=(A[i][j]-sum)/L[j][j]
    return L

In [ ]:

# =============================================
# Método de Ecuaciones Normales + Cholesky
# =============================================

def generar_datos(m=21, n=12, epsilon=1e-10):
    ##Genera matriz y vector y perturbarlo##
    # Generar puntos t_i
    t = np.linspace(0, 1, m)

    # Matriz de Vandermonde (m x n)
    A = np.array([[ti**j for j in range(n)] for ti in t])

    # Valores originales y_i (coeficientes 1)
    y_original = A @ np.ones(n)

    # Perturbación
    u = np.random.rand(m)
    y = y_original + (2*u - 1)*epsilon

    print(y_original)
    print(y)

    return A, y


def SolveChol(A,b):
    L=Cholesky(A)
    y = SustDelante(L, b)
    x = SustAtras(L.T, y)
    return x

# =============================================
# Método de Householder QR
# =============================================

def householder_qr(A):
    """Factorización QR usando reflectores de Householder"""
    m, n = A.shape
    R = A.copy().astype(float)
    Q = np.eye(m)

    for k in range(n):
        # Vector de la columna k
        x = R[k:, k]

        # Calcular reflector
        e1 = np.zeros_like(x)
        e1[0] = np.sign(x[0]) * LA.norm(x)
        v = x - e1
        v = v / LA.norm(v)  # Vector de Householder

        # Aplicar la reflexión a R
        R[k:, k:] -= 2 * np.outer(v, v @ R[k:, k:])

        # Almacenar transformaciones en Q (opcional para Q explícita)
        Q[k:] = Q[k:] - 2 * np.outer(v, v @ Q[k:])

    return Q.T, R[:n, :n]

def resolver_qr(A, y):
    """Resuelve mínimos cuadrados usando QR"""
    Q, R = householder_qr(A)
    y_proj = Q.T @ y
    x = LA.solve(R, y_proj[:n])  # Usar solo las primeras n filas
    return x

# =============================================
# Ejecución y Comparación
# =============================================
# Parámetros
m = 21
n = 12
epsilon = 1e-10

# Generar datos
A, y = generar_datos(m, n, epsilon)
print("========================")

# Método de Cholesky
x_cholesky = SolveChol(A.T@A, A.T@y)
print(f"Resultado con Cholesky: {x_cholesky}")
print("========================")
# Método QR
x_qr = resolver_qr(A, y)
print(f"Resultado con método House Holder QR: {x_qr}")

[ 1.          1.05263158  1.11111111  1.17647059  1.24999999  1.33333325
  1.42857067  1.53845634  1.6666387   1.81805645  1.99951172  2.22051952
  2.49455804  2.8408914   3.28719571  3.87329459  4.65640262  5.71838829
  7.17570464  9.19279825 12.        ]
[ 1.          1.05263158  1.11111111  1.17647059  1.24999999  1.33333325
  1.42857067  1.53845634  1.6666387   1.81805645  1.99951172  2.22051952
  2.49455804  2.8408914   3.28719571  3.87329459  4.65640262  5.71838829
  7.17570464  9.19279825 12.        ]
Resultado con Cholesky: [1.         0.99999309 1.00026581 0.99612653 1.02929549 0.86903618
 1.36885191 0.32746264 1.79254602 0.41735438 1.2429276  0.95614035]
Resultado con método House Holder QR: [1.         0.99999998 1.00000058 0.99999233 1.00005458 0.99976389
 1.00065654 0.99880172 1.00142639 0.99893478 1.00045262 0.99991659]




---


a.) Los resultados obtenidos nos sugieren que el método de Cholesky es más proponeso a tener fallas cuando el vector y varía por las perturbaciones generadas.

b.) El método que mejor aproxima la solución a $x_i=1$ es House Holder.

c.) Sí afecta, al moemnto de evalar los valores de $t_i$ obtendremos valores distintos en $y_i$, una variabilidad más alta aproxima mal los puntos.